### _Семенов Кирилл, БКЛ162_

In [34]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

from difflib import get_close_matches

import textdistance

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [35]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]


In [36]:
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [11]:
vec

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [37]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    candidates = [id2word[top] for top in topn]
    similarities = Counter()
    for word in candidates:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [38]:
%%time
get_closest_hybrid_match('Моксв', X, vec)

Wall time: 84.8 ms


('москва', 0.33333333333333337)

In [39]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))



In [41]:
correct = 0
total = 0
bad_corrections = []
for i in range(200):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        
        predicted = get_closest_hybrid_match(pair[1], X, vec)[0]
        if predicted == pair[0]:
            correct += 1
        else:
            bad_corrections.append((predicted,pair[0]))
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)

0
0.5333333333333333
10
0.7615384615384615
20
0.7748917748917749
30
0.7702020202020202
40
0.7680311890838206
50
0.7761674718196457
60
0.7666666666666667
70
0.7747524752475248
80
0.7736450584484591
90
0.7740667976424361
100
0.7724014336917563
110
0.7766582703610412
120
0.7811291569992266
130
0.7794871794871795
140
0.7778537252221462
150
0.7870722433460076
160
0.7871462264150944
170
0.7883211678832117
180
0.7868589743589743
190
0.7890781563126252


In [42]:
print('the ratio of correct corrections: '+str(correct/total))

the ratio of correct corrections: 0.7927710843373494


### смотрим на ошибки:

In [66]:
bad_corrections

[('симпатичный', 'симпатичнейшее'),
 ('шпионской', 'шпионское'),
 ('гламур', 'гламурный'),
 ('бонди', 'бонда'),
 ('superjet', 'superheadz'),
 ('ap', 'clap'),
 ('tamer', 'camera'),
 ('стояния', 'поясним'),
 ('ополчатся', 'получатся'),
 ('язычка', 'язычки'),
 ('милы', 'милые'),
 ('нащокина', 'насчет'),
 ('чавес', 'чавеса'),
 ('основа', 'основная'),
 ('попавшие', 'попавшим'),
 ('поисково-спасательных', 'аварийно-спасательных'),
 ('краснов', 'напрасно'),
 ('общем', 'в'),
 ('как', 'общем'),
 ('вы', 'как'),
 ('знаете', 'вы'),
 ('из', 'знаете'),
 ('моего', 'из'),
 ('не', 'моего'),
 ('давнего', 'недавнего'),
 ('пропал', 'пропажу'),
 ('щекам', 'ящика'),
 ('почте', 'почте.ру'),
 ('хорошей', 'хорошо'),
 ('выход', 'выходных'),
 ('крите', 'рите'),
 ('датам', 'потому'),
 ('штат', 'что'),
 ('упреждать', 'переждать'),
 ('дубрава', 'дубраве'),
 ('алюминия', 'люминала'),
 ('тыка', 'повтыкав'),
 ('летным', 'билетным'),
 ('касса', 'кассам'),
 ('тото', 'что-то'),
 ('тамошний', 'мощный'),
 ('рабочем', 'нера

### Метрика моего авторства - посмотреть среднее расстояние Левенштейна в ошибочно исправленных словах:

In [43]:
distances = 0
for pair in bad_corrections:
    d = textdistance.levenshtein(pair[0], pair[1])
    distances+=d

In [45]:
print('the average Levenstein distance of the wrong corrections: '+str(distances/len(bad_corrections)))

the average Levenstein distance of the wrong corrections: 3.2348837209302324


### Вывод:
неплохо, но можно и получше

## Подправляем н-граммы:

In [46]:
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,4))
X = vec.fit_transform(vocab)



In [48]:
correct = 0
total = 0
bad_corrections = []
for i in range(200):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        
        predicted = get_closest_hybrid_match(pair[1], X, vec)[0]
        if predicted == pair[0]:
            correct += 1
        else:
            bad_corrections.append((predicted,pair[0]))
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)

0
0.5333333333333333
10
0.7846153846153846
20
0.7922077922077922
30
0.7929292929292929
40
0.7992202729044834
50
0.8067632850241546
60
0.7986111111111112
70
0.801980198019802
80
0.8087141339001063
90
0.8094302554027505
100
0.8064516129032258
110
0.8094038623005877
120
0.811291569992266
130
0.8095238095238095
140
0.8092959671907041
150
0.8155893536121673
160
0.8154481132075472
170
0.815272318921954
180
0.8135683760683761
190
0.814629258517034


In [49]:
print('the ratio of correct corrections: '+str(correct/total))

the ratio of correct corrections: 0.8178313253012048


In [50]:
distances = 0
for pair in bad_corrections:
    d = textdistance.levenshtein(pair[0], pair[1])
    distances+=d

In [52]:
print('the average Levenstein distance of the wrong corrections: '+str(distances/len(bad_corrections)))

the average Levenstein distance of the wrong corrections: 2.5793650793650795


### Вывод:
Очень хорошее улучшение, целых два процента! И расстояние Левенштейна сократилось!

Но посмотрим на ошибки:

In [53]:
bad_corrections

[('симпатичный', 'симпатичнейшее'),
 ('шпионской', 'шпионское'),
 ('гламур', 'гламурный'),
 ('бонди', 'бонда'),
 ('superjet', 'superheadz'),
 ('ap', 'clap'),
 ('tamer', 'camera'),
 ('стояния', 'поясним'),
 ('ополчатся', 'получатся'),
 ('язычка', 'язычки'),
 ('милы', 'милые'),
 ('нащокина', 'насчет'),
 ('чавес', 'чавеса'),
 ('основа', 'основная'),
 ('попавшие', 'попавшим'),
 ('поисково-спасательных', 'аварийно-спасательных'),
 ('краснов', 'напрасно'),
 ('общем', 'в'),
 ('как', 'общем'),
 ('вы', 'как'),
 ('знаете', 'вы'),
 ('из', 'знаете'),
 ('моего', 'из'),
 ('не', 'моего'),
 ('давнего', 'недавнего'),
 ('пропал', 'пропажу'),
 ('щекам', 'ящика'),
 ('почте', 'почте.ру'),
 ('хорошей', 'хорошо'),
 ('выход', 'выходных'),
 ('крите', 'рите'),
 ('датам', 'потому'),
 ('штат', 'что'),
 ('упреждать', 'переждать'),
 ('дубрава', 'дубраве'),
 ('алюминия', 'люминала'),
 ('тыка', 'повтыкав'),
 ('летным', 'билетным'),
 ('касса', 'кассам'),
 ('тото', 'что-то'),
 ('тамошний', 'мощный'),
 ('рабочем', 'нера

Очень много чередований внутри словоформ одной лексемы, но бывает и путаница между частями речи. Попробуем это устранить.

## Подправляем код, добавив pymorphy:

In [54]:
import pymorphy2
from pymorphy2 import MorphAnalyzer


In [55]:
def pos_finder(w):
    morph = MorphAnalyzer()
    ana = morph.parse(w)
    pos = str(ana[0].tag).split(',')[0]
    return pos

def get_closest_hybrid_match_2(text, X, vec, metric=textdistance.levenshtein, TOPN=3):
    text_pos = pos_finder(text)
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    #print(topn)
    candidates = []
    pre_candidates = [id2word[top] for top in topn]
    for c in pre_candidates:
        c_pos = pos_finder(c)
        if c_pos == text_pos:
            candidates.append(c)
    if len(candidates) == 0:
        candidates = pre_candidates
    similarities = Counter()
    for word in candidates:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [57]:
correct = 0
total = 0
bad_corrections = []
for i in range(200):
    word_pairs = align_words(true[i], bad[i])
    #print(word_pairs)
    for pair in word_pairs:
        
        predicted = get_closest_hybrid_match_2(pair[1], X, vec)[0]
        if predicted == pair[0]:
            #print('good: '+predicted+' '+pair[0])
            correct += 1
        else:
            bad_corrections.append((predicted,pair[0]))
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)

0
0.5333333333333333
10
0.7769230769230769
20
0.7878787878787878
30
0.7904040404040404
40
0.797270955165692
50
0.8051529790660226
60
0.7972222222222223
70
0.7970297029702971
80
0.8023379383634431
90
0.8015717092337917
100
0.7983870967741935
110
0.8001679261125105
120
0.8020108275328693
130
0.8
140
0.7983595352016405
150
0.8054499366286438
160
0.8054245283018868
170
0.8051656372824256
180
0.8023504273504274
190
0.8041082164328658


In [58]:
print('the ratio of correct corrections: '+str(correct/total))

the ratio of correct corrections: 0.8067469879518072


In [59]:
distances = 0
for pair in bad_corrections:
    d = textdistance.levenshtein(pair[0], pair[1])
    distances+=d

print('the average Levenstein distance of the wrong corrections: '+str(distances/len(bad_corrections)))

the average Levenstein distance of the wrong corrections: 2.7231920199501247


### Вывод:
качество колеблется на том же уровне, даже немного похуже.